<a href="https://colab.research.google.com/github/aimbsg/EVA4_S8/blob/master/EVA4_S8_backbone_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Manual created model

In [0]:
import torch.nn as nn
import torch.nn.functional as F
drop_out = 0.05

class Net(nn.Module):
  def __init__(self):    
    super(Net, self).__init__()
    
    # Maxpooling
    self.pool = nn.MaxPool2d(2,2)

    # Input block
    self.conv1 = nn.Sequential(nn.Conv2d(3,16,3,1,1), nn.BatchNorm2d(16),nn.ReLU(), nn.Dropout(p=drop_out),
                                nn.Conv2d(16,32,3,1,1, dilation=1), nn.ReLU(), nn.BatchNorm2d(32),nn.Dropout(p=drop_out)
    )
    # Convolution block        
    self.conv2 = nn.Sequential(nn.Conv2d(32,64,3,1,1), nn.BatchNorm2d(64),nn.ReLU(), nn.Dropout(p=drop_out),
                                nn.Conv2d(64,128,3,1,1,dilation=2), nn.BatchNorm2d(128),nn.ReLU(),nn.Dropout(p=drop_out)
    )
    # Transition block
    self.convblock1 = nn.Sequential(nn.Conv2d(128,64,1), nn.BatchNorm2d(64), nn.ReLU())
    
    # Convolution block
    self.conv3 = nn.Sequential(nn.Conv2d(64,128,3,1,1), nn.BatchNorm2d(128),nn.ReLU(), nn.Dropout(p=drop_out),
                                nn.Conv2d(128,256,3,1,1), nn.BatchNorm2d(256),nn.ReLU(), nn.Dropout(p=drop_out)
    )        
    # Transition block
    self.convblock2 = nn.Sequential(nn.Conv2d(256,64,1), nn.BatchNorm2d(64), nn.ReLU())  
    
    # Convolution block      
    self.conv4 = nn.Sequential(nn.Conv2d(64,128,3,1,1, dilation = 1), nn.BatchNorm2d(128),nn.ReLU(), nn.Dropout(p=drop_out)
    )
    # Transition block
    # self.convblock3 = nn.Sequential(nn.Conv2d(128,32,3,1,1), nn.ReLU(), nn.BatchNorm2d(32))
    
    # Depthwise block
    self.dwconv = nn.Sequential(nn.Conv2d(128,128,3,padding=1,groups=128), 
    nn.Conv2d(128,256,1),nn.BatchNorm2d(256), nn.Dropout(p=drop_out), nn.ReLU()
    )
    # Output classes block
    # self.convblock4 = nn.Sequential(nn.Conv2d(128,10,1), nn.ReLU(), nn.BatchNorm2d(10))
    self.convblock3 = nn.Sequential(nn.Conv2d(256,10,1), nn.BatchNorm2d(10), nn.ReLU())

    # GAP block
    self.gap = nn.AvgPool2d(kernel_size = 3)

  def forward(self, x):    
    x = self.pool(self.conv1(x))
    # print("1")
    # print(x.shape)
    x = self.convblock1(self.pool(self.conv2(x)))        
    # print("2")
    # print(x.shape)
    x = self.convblock2(self.pool(self.conv3(x)))
    # print("3")
    # print(x.shape)
    # x = self.conv4(x)        
    # x = self.convblock3(x)
    # x = self.pool(x)
    x = self.convblock3(self.dwconv(self.conv4(x)))
    # print("4")
    # print(x.shape)
    x = self.gap(x)
    
    x = x.view(-1, 10*1*1)
    return F.log_softmax(x, dim=-1)

net = Net()        


In [0]:
def train(model, device,trainloader,optimizer,criterion,epoch):
  running_loss = 0.0

  for i, data in enumerate(trainloader, 0):
    # get the inputs
    inputs, labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    if i % 2000 == 1999:    # print every 2000 mini-batches
        print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_loss / 2000))
        running_loss = 0.0


In [0]:
def val_acc(testloader,net,device):
  correct = 0
  total = 0
  with torch.no_grad():
      for data in testloader:
          images, labels = data[0].to(device), data[1].to(device)
          outputs = net(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  print('Accuracy of the network on the 10000 test images: %2f %%' % (
      100 * correct / total))


In [0]:
def class_acc(testloader,net,device,classes):
  class_correct = list(0. for i in range(10))
  class_total = list(0. for i in range(10))
  with torch.no_grad():
      for data in testloader:
          images, labels = data[0].to(device), data[1].to(device)
          outputs = net(images)
          _, predicted = torch.max(outputs, 1)
          c = (predicted == labels).squeeze()
          for i in range(4):
              label = labels[i]
              class_correct[label] += c[i].item()
              class_total[label] += 1

  for i in range(10):
      print('Accuracy of %5s : %2d %%' % (
          classes[i], 100 * class_correct[i] / class_total[i]))

In [0]:
def save_ckp(state, checkpoint_path):
    """
    state: checkpoint we want to save
    checkpoint_path: path to save checkpoint
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)

In [0]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min